In [1]:
import pandas as pd
from src.training.longformer_pipeline import LongFormerPipeline
from src.training.bert_pipeline import TrainingBertPipeline
import logging
import torch
import os

In [2]:
df = pd.read_csv("data/full_aes_dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22406 entries, 0 to 22405
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   question           4859 non-null   object 
 1   reference_answer   22406 non-null  object 
 2   answer             22406 non-null  object 
 3   score              22406 non-null  float64
 4   dataset            22406 non-null  object 
 5   normalized_score   22406 non-null  float64
 6   normalized_score2  22406 non-null  int64  
 7   bert_length        22406 non-null  int64  
 8   indobert_length    22406 non-null  int64  
 9   albert_length      22406 non-null  int64  
 10  longformer_length  22406 non-null  int64  
 11  multibert_length   22406 non-null  int64  
 12  indoalbert_length  22406 non-null  int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 2.2+ MB


In [3]:
results = []
results_epoch = []
batch_sizes = [2, 4]
epochs_list = [5, 10]
learning_rates = [1e-5, 2e-5, 5e-5]
idx = 0  # index untuk setiap kombinasi

ROOT_DIR = os.getcwd()

In [ ]:
for batch_size in batch_sizes:
    for num_epochs in epochs_list:
        for lr in learning_rates:
            config = {
                "df": df,
                "model_name": "allenai/longformer-base-4096",
                "batch_size": batch_size,
                "learning_rate": lr,
                "epochs": num_epochs,
                "config_id": idx,
                "max_seq_len": 2048,
            }

            logging.info(
                f"Running configuration: config_id={idx}, model_name={config['model_name']}, "
                f"batch_size={batch_size}, epochs={num_epochs}, learning_rate={lr}"
            )
            print(
                f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, "
                f"batch_size={batch_size}, epochs={num_epochs}, learning_rate={lr}"
            )
            
            try:
                pipeline = LongFormerPipeline(config, results, results_epoch)
                pipeline.run_training()

                # Save results
                # Dapatkan root project
                results_path = os.path.join(ROOT_DIR, "experiments/results/results_longformer.csv")
                results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch_longformer.csv")
                TrainingBertPipeline.save_csv(results, results_path)
                TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
            except Exception as e:
                logging.error(f"Error in config_id={idx}: {str(e)}")
                print(f"Error in config_id={idx}: {str(e)}")
                torch.cuda.empty_cache()
            finally:
                # Clear GPU memory after every configuration
                del pipeline.model
                del pipeline.tokenizer
                del pipeline.optimizer
                torch.cuda.empty_cache()
            idx += 1


Running configuration: config_id=0, model_name=allenai/longformer-base-4096, batch_size=2, epochs=1, learning_rate=1e-05
split dataset run...
create dataset run...
create dataloader run...
create dataloader done...
====== Training Epoch 1/1 ======
